In [1]:
import pyodbc

In [2]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


## SQL Server Connection

In [3]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

Asıl mevzumuz pyodbc metodunu kullanarak bir connect oluşturmak. İstenilenleri ister bunun içine yazmak ya da dışarda oluşturup aşağıda yazmak. Buradaki fonksiyon aslında connection un başarılı olup olmadığını kontrol etmek.

In [4]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'master'#yeni bir db oluşturacağız fakat mevcut bir database'e bağlanacaksak onu yazarız.
user = 'sa'
password = 'Emrhn1031.'
conn = create_server_connection(driver, server, database, user, password)

Successfull


### windows authentication

Burada yaptığımız ise bir trusted connection oluşturmak.

In [167]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [168]:
conn = pyodbc.connect(conn_string);  # connection objesi

In [169]:
crs = conn.cursor()  # database içindeki verilere satır satır ulaşmayı sağlayan araçtır

In [170]:
conn.autocommit = True # conn.commit() we did automatically

In [76]:
create_database_query = 'Create database test_1'

In [10]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try:
        crs.execute(create_database_query)
        print("Database is created succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [11]:
create_database(conn, create_database_query)

Database is created succesfully


#### aslında bu aşağıdaki aynı yukarıda olduğu gibidir.

Biz bu execute query i kullanarak use test1 i çalıştıracağız.

In [163]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [165]:
query = 'USE test_1'
execute_query(conn,query)

Query Succeessful!


## Create Table

Şifre girmemek için Windows Authenticaion ile bağlandık

In [14]:
query = 'CREATE TABLE TestA (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'
execute_query(conn, query)

Query Succeessful!


artık test1 database i altında TestA tablosu oluşturdum.

## Insert (tabloya veri insert etme)

cursor üzerinde execute ile alttaki syntaxımızı çalıştıracağız. Her seferinde commit yapmak zorundaydık ama biz auto cmmit yaptığımız için ***conn.commit()*** e gerek yok

In [15]:
crs = conn.cursor()
crs.execute("INSERT TestA(FirstName,LastName) VALUES('Bob', 'Marley')")
#conn.commit()

***Bob Marley insert edilmiş oldu***

In [16]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

In [17]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')

In [18]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [19]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')

In [20]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')

In [21]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')

In [22]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## Delete

In [23]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Jerry'")  # first name i Jerry olanı id si ile birlikte siler

In [24]:
crs.execute("DELETE FROM TestA WHERE FirstName = 'Bob'") 

In [25]:
crs.execute("DELETE FROM TestA WHERE FirstName = ?" ,'Stefan')

## Rowcount 

Son yapılan execute işlem sonucunda etkilenen işlem sayısını gösterir.

In [26]:
crs.rowcount

1

In [27]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


## Read Data

## fetchone()

Sql deki verileri database e gitmeden verileri yakalayıp görüntülemeye yarar.

In [28]:
crs.execute('SELECT FirstName, LastName FROM TestA')
row = crs.fetchone() # tablodaki ilk kaydı getirir
row

('Bruce', 'Lee')

In [45]:
row = crs.fetchone() # artık ilk veriyi atlar ve ikinci kaydı getirdi
row

('Tom', 'Hanks')

In [32]:
crs.execute('SELECT FirstName, LastName FROM TestA')
row = crs.fetchone() 
row

('Bruce', 'Lee')

eğer atlayıp ikinciden gitmesini değil en baştan başlamasını isterseniz en başına tekrardan şunu yazmak gerek.

***crs.execute('SELECT FirstName, LastName FROM TestA')***

In [46]:
row.FirstName  # cursorun mevcut olduğu satırdaki istenen column verisini getirdi

'Tom'

In [53]:
row.LastName  # cursorun mevcut olduğu satırdaki istenen column verisini getirdi

'Hanks'

## fetchall()

In [54]:
crs.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = crs.fetchall()
rows

[(2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [55]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Cat
7 Owen William


3 tırnak işareti ile önceden kullandığımız kodu tekrar çağırabiliriz) Aşağıdaki sorgu şunla aynıdır= ***SELECT * FROM TestA***

In [56]:
crs.execute("""SELECT ID, FirstName, LastName
                FROM TestA""")
rows = crs.fetchall()
rows

[(2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [57]:
type(rows)  # datatype leara bakar

list

In [58]:
# filtreleme yapabiliriz.

crs.execute("""SELECT ID, FirstName, LastName
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(7, 'Owen', 'William')]

## fetchmany()

İstediğimiz kadar, döndürmeyi sağlar

In [62]:
crs.execute("""SELECT ID, FirstName, LastName
               FROM TestA""")
rows = crs.fetchmany(5)
rows

[(2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

## Fetchval()

değeri yani value yi yakalar

In [63]:
crs.execute("SELECT MAX(ID) FROM TestA").fetchval()

7

In [65]:
crs.execute("SELECT LastName FROM TestA").fetchval()

'Lee'

## Pandas Methods

In [66]:
import pandas as pd

Burada pandas'ın read Sql metodunu göreceğiz.

In [85]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [86]:
query = """SELECT ID, FirstName, LastName
               FROM TestA"""

In [70]:
# crs = conn.cursor()

In [87]:
result = read_query (conn, query)

In [88]:
from_db = []
for result in result:
    result = list(result)
    from_db.append(result)

In [89]:
columns = ['ID','FirstName', 'LastName']
df = pd.DataFrame(from_db, columns = columns)
df

,ID,FirstName,LastName
0,2,Bruce,Lee
1,3,Tom,Hanks
2,4,Bart,Simpson
3,5,Tom,Cat
4,7,Owen,William


In [90]:
query = 'USE BikeStores'
execute_query(conn,query)

Query Succeessful!


In [91]:
crs.execute("SELECT * FROM production.brands").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [92]:
pd.read_sql("SELECT * FROM production.brands", con = conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [93]:
df_prod = pd.read_sql("SELECT * FROM production.products", con = conn)
df_prod.head(10)

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,8,6,2016,469.99
6,7,Trek Slash 8 27.5 - 2016,9,6,2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,9,6,2016,1799.99
8,9,Trek Conduit+ - 2016,9,5,2016,2999.99
9,10,Surly Straggler - 2016,8,4,2016,1549.00


## DF to Database

In [94]:
import sqlalchemy
import urllib

In [171]:
create_database_query = 'Create database Emirhan'

In [172]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try:
        crs.execute(create_database_query)
        print("Database is created succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [173]:
create_database(conn, create_database_query)

Database is created succesfully


In [174]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [175]:
crs.execute('USE Emirhan')

In [176]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' #şu anda server her birimizin kendi makinesi
database = 'Emirhan'#yeni bir db oluşturacağız
user = 'sa'
password = 'Emrhn1031.' 

In [177]:
params = 'DRIVER='+driver + ';SERVER=' + server +';PORT=1433;DATABASE='+ database + ';UID=' + user + ';PWD=' + password

In [178]:
db_params = urllib.parse.quote_plus (params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [179]:
df_prod = pd.read_sql("SELECT * FROM production.products where product_id <10", con = conn)
df_prod.head(10)

DatabaseError: Execution failed on sql 'SELECT * FROM production.products where product_id <10': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'production.products'. (208) (SQLExecDirectW)")

In [180]:
df_prod.to_sql("products_bikes", engine, index = False, if_exists = "append", schema = "dbo")